# Darts?

<!DOCTYPE html>
<html>
<head>
    <style>
        .image-container {
            display: flex; /* Flex 컨테이너 설정 */
        }
        .image-container img {
            width: 50%; /* 이미지 너비 설정 */
            height: auto; /* 이미지 높이 자동 조정 */
        }
    </style>
</head>
<body>

<div class="image-container">
    <img src="https://unit8.com/wp-content/uploads/2021/07/image-13.png" alt="Image 1">
</div>

</body>
</html>


**Darts**는 Python을 위한 오픈 소스 라이브러리로, 시계열 데이터에 대한 예측 및 분석을 수행하기 위해 설계되었습니다.
시계열 예측을 위한 다양한 모델과 도구를 제공하며, 사용하기 쉽고 유연한 API를 갖추고 있습니다.

- **다양한 시계열 모델 지원**: Darts는 전통적인 통계 모델(예: ARIMA, Exponential Smoothing)부터 최신 딥 러닝 모델(예: LSTM, Transformer, DLinear, N-HITS. . . )에 이르기까지 다양한 시계열 예측 모델을 지원합니다.
- **시계열 데이터 처리**: 시계열 데이터를 쉽게 처리하고 변환할 수 있는 기능을 제공합니다. 예를 들어, 시계열을 훈련 및 테스트 세트로 분할하거나, 시계열을 스케일링하고, 특성을 추출하는 등의 작업을 수행할 수 있습니다.
- **모델 훈련 및 예측**: Darts는 모델을 훈련하고 미래의 데이터 포인트를 예측하는 간단한 인터페이스를 제공합니다. 이를 통해 사용자는 적은 코드로 모델을 빠르게 훈련시키고 예측을 생성할 수 있습니다.
- **모델 평가**: 다양한 평가 지표를 사용하여 모델의 성능을 측정하고 비교할 수 있습니다.
- **시각화 도구**: 시계열 데이터와 예측 결과를 시각화하기 위한 도구를 제공합니다. 이를 통해 데이터의 패턴을 쉽게 파악하고 모델의 성능을 직관적으로 이해할 수 있습니다.
- **확장성**: Darts는 사용자가 자신만의 모델을 정의하고 기존 모델과 함께 사용할 수 있도록 확장 가능한 구조를 가지고 있습니다.

# Description

- 2019.01.01 ~ 2023.03.03까지의 유통 가격 데이터
- `item` : 품목
   - TG : 감귤 (Tangerine)
   - BC : 브로콜리 (Broccoli)
   - RD : 무 (Radish)
   - CR : 당근 (Carrot)
   - CB : 양배추 (Cabbage)
- `corporation` : 유통 법인 코드
  - 법인 A부터 F 존재
- `location` : 지역 코드
  - J : 제주도 제주시
  - S : 제주도 서귀포시
- `supply(kg)` : 유통된 물량, kg 단위
- `price(원/kg)` : 유통된 품목들의 kg 마다의 가격, 원 단위

In [1]:
# !pip install pytimekr darts

In [4]:
#Regression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold, cross_val_score, train_test_split, TimeSeriesSplit
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, VotingRegressor, StackingRegressor, AdaBoostRegressor, HistGradientBoostingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, f1_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder,OrdinalEncoder, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.inspection import PartialDependenceDisplay
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# Data

In [5]:
import os
os.getcwd()

'/Users/ktg/Desktop/Jeju-Specialties'

In [6]:
path = os.getcwd()
train = pd.read_csv(path + "/train.csv")
test = pd.read_csv(path + "/test.csv",index_col='timestamp', parse_dates=True)
ss = pd.read_csv(path + "/sample_submission.csv")
it = pd.read_csv(path + "/international_trade.csv")

# Model

In [7]:
#쥬혁이님 코드 활용
darts_data = train.rename(columns={'timestamp':'ds', 'price(원/kg)':'y'})
darts_data = darts_data[['ID','ds','y']]
darts_data['ID'] = darts_data['ID'].str.replace(r'_\d{8}$','', regex=True)

In [8]:
darts_data.head()

,ID,ds,y
0,TG_A_J,2019-01-01,0.0
1,TG_A_J,2019-01-02,0.0
2,TG_A_J,2019-01-03,1728.0
3,TG_A_J,2019-01-04,1408.0
4,TG_A_J,2019-01-05,1250.0


# Final

In [9]:
from darts.timeseries import TimeSeries
from darts.models import DLinearModel

random_seed = 42
np.random.seed(random_seed)

In [10]:
code = darts_data['ID'].unique()[1]
d = darts_data[darts_data['ID'] == code].reset_index().drop(['ID'], axis=1).sort_values('ds')
ts = TimeSeries.from_dataframe(d[['y']].astype('float32'))

In [14]:
darts_data

,ID,ds,y
0,TG_A_J,2019-01-01,0.0
1,TG_A_J,2019-01-02,0.0
2,TG_A_J,2019-01-03,1728.0
3,TG_A_J,2019-01-04,1408.0
4,TG_A_J,2019-01-05,1250.0
...,...,...,...
59392,RD_F_J,2023-02-27,468.0
59393,RD_F_J,2023-02-28,531.0
59394,RD_F_J,2023-03-01,574.0
59395,RD_F_J,2023-03-02,523.0


In [11]:
d

,index,ds,y
time,,,
0,1523,2019-01-01,0.0
1,1524,2019-01-02,0.0
2,1525,2019-01-03,2526.0
3,1526,2019-01-04,2134.0
4,1527,2019-01-05,2075.0
...,...,...,...
1518,3041,2023-02-27,4166.0
1519,3042,2023-02-28,4009.0
1520,3043,2023-03-01,4173.0


In [1]:
from darts.timeseries import TimeSeries
from darts.models import DLinearModel

random_seed = 42
np.random.seed(random_seed)

#쥬혁이님 코드 활용
def ph_train(df):
    pred_list = []
    for code in darts_data['ID'].unique():
        d = darts_data[darts_data['ID'] == code].reset_index().drop(['ID'], axis=1).sort_values('ds')

        ts = TimeSeries.from_dataframe(d[['y']].astype('float32'))

        model = DLinearModel(input_chunk_length=128, output_chunk_length=28, batch_size=256, n_epochs=50)
        model.fit(ts)
        
        future = pd.DataFrame()
        future['ds'] = pd.date_range(start='2023-03-04',periods=28,freq='D') #prophet_data['ID'].nunique() = 39
        forecast = model.predict(len(future['ds']))
        pred_y = forecast.values()
        pred_code = [str(code)] * len(pred_y)
        for y_val, id_val in zip(pred_y, pred_code):
            pred_list.append({'ID':id_val, 'y':y_val})

    pred = pd.DataFrame(pred_list)
    
    return pred

pred = ph_train(darts_data)


KeyboardInterrupt



In [21]:
pred['y'].apply(lambda x: len(x))

0       1
1       1
2       1
3       1
4       1
       ..
1087    1
1088    1
1089    1
1090    1
1091    1
Name: y, Length: 1092, dtype: int64

In [23]:
pred['y'] = pred['y'].explode().astype(float)
pred['y']

0       2871.312500
1       1182.883789
2       2571.392578
3       2679.567139
4       2485.250000
           ...     
1087     380.496460
1088     360.578796
1089     390.622070
1090     378.975342
1091     370.006836
Name: y, Length: 1092, dtype: float64

In [24]:
pred['y'][pred['y'] < 0]

169    -1357.884521
171     -191.247253
172     -398.636292
173      -52.281006
176     -886.060791
           ...     
945      -42.356136
947      -41.732437
949      -10.884001
950     -110.878387
1079      -0.216888
Name: y, Length: 122, dtype: float64

# submission

In [15]:
ss.loc[:,'answer'] = pred['y'].clip(0,)
path = "./DLinear_50_epochs.csv"
ss.to_csv(path,index=False)

In [41]:
!pip install dacon_submit_api-0.0.4-py3-none-any.whl

Processing ./dacon_submit_api-0.0.4-py3-none-any.whl


In [74]:
API = "API"

In [75]:
from dacon_submit_api import dacon_submit_api

result = dacon_submit_api.post_submission_file(
path, API,
'236176',
'NAMJIN',
path)

{'isSubmitted': True, 'detail': 'Success'}
